In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import pandas as pd

import pickle, os, sys

# imagebind
from models.constantinople import Constantinople
from tqdm import tqdm
from utils.custom_logger import get_logger
from utils.model_controler import freeze_model, run_train_test_2_input, evaluate_2_input

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
dataset = torch.load('dataset/constantinople_dataset.pth')

In [ ]:
train_size = int(len(dataset)*0.8)
test_size = len(dataset)-train_size

train, test = random_split(dataset, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

In [ ]:
model = Constantinople()
model.load_state_dict(torch.load('.checkpoints/pretrained_constantinople_512.pth'))

In [ ]:
freeze_model(model, 
             exception = ['postprocess', 'polling'])

model.train()
model.to(device)

In [ ]:
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 1
test_step = 1
logger = get_logger('autoencoder_portion')

In [ ]:
pred, target = evaluate_2_input(model, test_loader, criterion)

In [ ]:
logger.info('start_new_run--------------------------------')

result = run_train_test_2_input(
    model=model, 
    train_loader=train_loader, 
    test_loader=test_loader,
    best_test_loss = float('inf'),
    criterion=criterion, 
    optimizer=optimizer, 
    num_epochs=num_epochs, 
    test_step=test_step, 
    logger = logger, 
    device = device,
    path = '.checkpoints/best/venezia_best.pth')